In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

resultados = pd.DataFrame({
    'Modelo':[],
    'Detalles':[],
    'R2 train':[],
    'R2 test':[],
    'MAE train':[],
    'MAE test':[]
})

def error_absoluto_medio(y,y_pred):
    v = np.abs(y - y_pred)
    return v.sum()/len(y)
    
def error_gen(y_train,y_pred_train,y_test,y_pred_test):
    MAE_TRAIN = error_absoluto_medio(y_train,y_pred_train)
    MAE_TEST = error_absoluto_medio(y_test,y_pred_test)
    return MAE_TRAIN,MAE_TEST

## Lectura e inspección del dataset

El dataset utilizado no es uno de los más grandes que se pueden encontrar con lo que respecta al precio de venta de casas. No obstante, aprovecharemos la baja cantidad de caracteristicas recopiladas para graficar su relación con la variable a predecir, en este caso el precio; y la relacion entre las mismas. Antes de optar por escoger un algoritmo de regresión, la mejor opcion es dar un vistaso al dataset y analizar el comportamiento de las variables.

In [ ]:
df = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')
df.drop(['id'],axis=1,inplace=True)

df.head()

### Valores faltantes

Dado que el dataset ha sido acondicionado previamente para su uso, será raro encontrar valores faltantes o valores atápicos(outliers) pero nunca está de más asegurarnos de que los datos están completos.

In [ ]:
df.isna().sum()

### Visualizacion de datos

Partiremos con la visualizacion de los histogramas de cada caracteristica del dataset. En la gráfica se observa que <b>price</b>, <b>sqft_living</b>, <b>sqft_above</b> y <b>sqft_living15</b> presentan skew positivo. Esta información será util para determinar que variables serán transformadas para que su distribucion se ajuste a la de la normal y, talvez, ayude a mejorar los resultados de la predicción.

In [ ]:
from plotly.subplots import make_subplots
import numpy as np

xpoint, ypoint = np.meshgrid(range(4),range(5),indexing='ij')
cols = df.columns.values

fig2 = make_subplots(rows=4,cols=5)
for i,j,col in zip(xpoint.ravel(),ypoint.ravel(),cols):
    fig2.add_histogram(x=df[col],name=col,row=i+1,col=j+1)
    
fig2.show()

El dataset utilizado nos proporcina los datos de longitud y latitud de cada casa. Sería un despercidio no utilizarlos para visualizar su posición en un mapa real. Dado esto gracias al token obtenido de <b>MapBox</b> obtenemos un mapa de las casas descritas en el dataset y como están agrupadas en relación a su código zip.

In [ ]:
mapbox_access_token = "pk.eyJ1IjoibGFlY3MiLCJhIjoiY2tkZ3RveWozMjUzNTJ3anFxaGNydnpvYyJ9.Qv6zEMCrHgxm01Xqn4L8gw"
px.set_mapbox_access_token(mapbox_access_token)

fig = px.scatter_mapbox(df, lat="lat", lon="long",color="zipcode",
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        title='Mapa organizado por código zip',
                        size_max=15, zoom=8.8,center=dict(lat=47.46,lon=-121.9))

fig.update_layout(
    autosize=False,
    height=800,
    width=1000,
    hovermode='closest',
)

fig.show()

Otra forma de aprovechar los datos de longitud y latitud es observar si estos afectan al precio de la casa. Una forma muy visual puede ser encontrando el precio promedio de cada zona urbana y ordenandolos. De esta forma obtenemos un mapa en el cual el precio promedio de las casas de una zona es mayor mientras más rojo sea y menor mientras más azul sea. Así podemos afirmar:

* La ubicación de la casa si influye en su precio en la mayoria de los caos, siendo los que presentan mayor latitud los que tienen mayor valor.

* Las zonas con mayor costo promedio son aquellas que se encuentran mas cercanas a fuentes naturales de agua tales como  lagos, rios o salidas al mar. Si se contara con la caracteristica de cercania a corrientes de agua seria un indicador que ayudaria a predecir el precio de la vivienda.


In [ ]:
mean_zip = df.groupby(['zipcode']).mean()['price'].sort_values(ascending=True)
code_zip = mean_zip.index.values.tolist()
val_zip = range(mean_zip.shape[0])

color = []
for code in df['zipcode']:
    ind = code_zip.index(code)
    color.append(val_zip[ind])

df['mean_price_sort'] = color

fig = px.scatter_mapbox(df, lat="lat", lon="long",color="mean_price_sort",
                        color_continuous_scale=px.colors.cyclical.IceFire, 
                        title='Mapa organizado por el precio medio de la zona',
                        size_max=15, zoom=8.8,center=dict(lat=47.46,lon=-121.9))

fig.update_layout(autosize=False,
                  height=800,
                  width=1000,
                  hovermode='closest')

fig.show()

df.drop('mean_price_sort',axis=1,inplace=True)

Cuando se explora un dataset es importante conocer como se relacionan los datos con la variable a predecir ya que esto ayudará a determinar cuales son las que ayudarán a que el modelo realice las mejores predicciones.  En este caso <b>bedrooms</b> y <b>bathrooms</b> parecen tener una relacion aparentemente lineal con <b>price</b>.  Tambien que parece existir una relación lineal entre <b>bathrooms</b> y <b>bedrooms</b>.

In [ ]:
fig = px.scatter_matrix(df,
                       dimensions=['price','bedrooms','bathrooms','floors','waterfront']
                       )
fig.show()

Otra de las caracteristicas que parecen tener una relacion lineal son <b>sqft_living</b> y <b>sqft_above</b>.

In [ ]:
fig = px.scatter_matrix(df,
                       dimensions=['price','sqft_living','sqft_living15','sqft_lot','sqft_lot15','sqft_above','sqft_basement']
                       )
fig.show()

En este caso en particular encontramos que <b>grade</b> y <b>price</b> poseen una relacion polinomial.

In [ ]:
fig = px.scatter_matrix(df,
                       dimensions=['price', 'view', 'condition', 'yr_built', 'yr_renovated', 'grade']
                       )
fig.show()

Dado que en el dataset estamos trabajando con 19 características, graficar la relación entre las mismas puede resultar una tarea excesiva por lo que es mejor mostrar solo aquellas que podrían tener una relación entre ellas. Para conocer como se relacionan todas las caracteristicas una opción más visual es realizar una matriz de correlación.

In [ ]:
matrix_corr = df.corr()

heat = go.Heatmap(z=matrix_corr.values,
                  x=matrix_corr.index.values,
                  y=matrix_corr.columns.values)

layout = go.Layout(title='Matriz de correlación',
                   width=800, height=800,
                   xaxis_showgrid=False,
                   yaxis_showgrid=False,
                   yaxis_autorange='reversed')

fig=go.Figure(data=[heat],layout=layout)        
fig.show()

En la matriz de correlacion podemos observar que aquellas caracteristicas que identificamos que poseen una aparente relacion lineal con el precio tambien poseen una gran correlacion. Las carecteristicas identificadas fueron : <b>bathrooms</b>, <b>bedrooms</b>, <b>grade</b>, <b>sqft_living</b> y <b>sqft_above</b>.

La finalidad de la matriz de correlacion no solo es identificar a aquellas caracteristicas que estan más relacionadas con la variable a predecir, sino tambien es identificar aquellas caracteristicas que presentan una alta correlación con otras. Proporcionar al modelo de regresión una gran cantidad de características no siempre es lo adecuado. la existencia de una alta correlacion entre dos características  puede ocasionar problemas de <b>overfitting</b> y fallos en las predicciones.

### Otras visualizaciones

Anteriormente hicimos un gráfico de dispersion par poder observa la relación entre las caracteristicas pero esta forma de representación no resulta ser la más simple de comprender si lo que se busca es que el lector entienda la intención del autor.
De esta forma utilizando un diagrama de caja  y bigote para cada caracteristica podemos apreciar como aumenta el precio de la casa mientras más baños y dormitorios tiene, y también mientras mejor sea su calidad(<b>grade</b>). 


In [ ]:
fig3 = make_subplots(rows=1,cols=2)
fig3.add_box(x=df['bathrooms'],y=df['price'],row=1,col=1,name='bathroom')
fig3.add_box(x=df['bedrooms'],y=df['price'],row=1,col=2,name='bedrooms')
fig3.show()

fig4 = px.box(df,x='grade',y='price',color='grade',title='Grade Box Plot')
fig4.show()

Tomando como ejemplo el mundo real, si observáramos una sombra circular no habría forma de saber si el cuerpo que proyecta dicha sombra es una esfera o un cilindro. De esta misma forma podriamos estar obviando alguna relación especial entre las caracteristicas ya que estas se encuentran en N-dimensiones. Dado que nosotros solo podemos hacer representaciones en 3D, aprovecharemos esto para explorar más a fondo los datos y así poder encontrar resultados que se obviaron en las gráficas 2D.  

In [ ]:
fig = px.scatter_3d(df, x='price', y='sqft_living', z='sqft_above',color='grade')
fig.show()

## Preprocesamiento de los datos

Antes de ingresar los datos a un modelo, estos deben ser acondicionados para que las predicciones sean las mas acertadas posibles. Para ello se utilizan técnicas como transformaciones, agrupamiento o normalización. Pero antes de realizar el preprocesamiento tengamos una idea de cual sería el resultado de ingresar los datos sin acondicionar a una regresión múltiple.  

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df2 = df.drop(['date'],axis=1)

def regresion_lineal(df,test_size=0.2,Prec_var='price'):
    x = df.drop(Prec_var,axis=1)
    y = df[Prec_var]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=test_size)
    lr = LinearRegression()
    lr.fit(x_train,y_train)

    r2_train = lr.score(x_train,y_train)
    r2_test = lr.score(x_test,y_test)
    
    y_train_pred = lr.predict(x_train)
    y_test_pred = lr.predict(x_test)
    
    MAE_train,MAE_test = error_gen(y_train,y_train_pred,y_test,y_test_pred)
    
    return r2_train,r2_test,MAE_train,MAE_test

r2_train,r2_test,MAE_train,MAE_test = regresion_lineal(df2)

final = resultados.shape[0]
resultados.loc[final] = ['Regresión Lineal','Sin procesar',r2_train,r2_test,MAE_train,MAE_test]
resultados

### Escalación de los datos

In [ ]:
from sklearn.preprocessing import MinMaxScaler
aux1 =  df2.drop('price',axis=1)
col_esc = aux1.columns.values

scale = MinMaxScaler()

df3 = df2
df3[col_esc] = scale.fit_transform(df2[col_esc])

In [ ]:
df3.head()

### Transformación no linear

#### Mapeo a distribución gausiana

In [ ]:
from sklearn.preprocessing import PowerTransformer

var_label1 = ['sqft_living', 'sqft_above', 'sqft_living15']
transf = PowerTransformer(method='box-cox',standardize=False)

var_transf = transf.fit_transform(df[var_label1])

df_tf = df3
df_tf[var_label1] = var_transf

xpoint, ypoint = np.meshgrid(range(1),range(3),indexing='ij')
fig2 =  make_subplots(rows=1,cols=3)
for i,j,col in zip(xpoint.ravel(),ypoint.ravel(),var_label1):
    fig2.add_histogram(x=df_tf[col],name=col,row=i+1,col=j+1)

fig2.update_layout(title_text="Histograma de las variables transformadas")
fig2.show()

In [ ]:
r2_train,r2_test,MAE_train,MAE_test = regresion_lineal(df_tf)

final = resultados.shape[0]
resultados.loc[final] = ['Regresión Lineal','Transf. a gausiana',r2_train,r2_test,MAE_train,MAE_test]
resultados

### Agrupamiento

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

var_label2 = ['yr_built','yr_renovated']
agrup = KBinsDiscretizer(n_bins=7, encode='ordinal', strategy='uniform')

var_bin = agrup.fit_transform(df[var_label2])

df_bin = df3
df_bin[var_label2] = var_bin

xpoint, ypoint = np.meshgrid(range(1),range(2),indexing='ij')
fig2 =  make_subplots(rows=1,cols=2)
for i,j,col in zip(xpoint.ravel(),ypoint.ravel(),var_label2):
    fig2.add_histogram(x=df_bin[col],name=col,row=i+1,col=j+1)
fig2.show()

In [ ]:
r2_train,r2_test,MAE_train,MAE_test = regresion_lineal(df_bin)

final = resultados.shape[0]
resultados.loc[final] = ['Regresión Lineal','Agrupamiento',r2_train,r2_test,MAE_train,MAE_test]
resultados

### Juntando ambas transformaciones

In [ ]:
df_tf_bin = df3
df_tf_bin[var_label1] = var_transf
df_tf_bin[var_label2] = var_bin

r2_train,r2_test,MAE_train,MAE_test = regresion_lineal(df_tf_bin,test_size=0.2)

final = resultados.shape[0]
resultados.loc[final] = ['Regresión Lineal','Agrup. y transf.',r2_train,r2_test,MAE_train,MAE_test]
resultados

## Selección del modelo de regresión

Al usar la regresion lineal obtenemos un porcentaje de precisión del 76%. Este resultado no está mal pero dado que el dataset posee varias características que no presentan una relación lineal con el precio, probaremos con modelos más flexibles.

### Regresión polinómica

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

def regresion_poli(df,degree=2,test_size=0.2,Prec_var='price'):
    x = df.drop(Prec_var,axis=1)
    y = df[Prec_var]
    
    poly = PolynomialFeatures(degree=2,)
    x_poly = poly.fit_transform(x)
    
    x_train,x_test,y_train,y_test = train_test_split(x_poly,y,test_size=test_size)
        
    lr = LinearRegression()
    lr.fit(x_train,y_train)
    
    r2_train = lr.score(x_train,y_train)
    r2_test = lr.score(x_test,y_test)
    
    y_train_pred = lr.predict(x_train)
    y_test_pred = lr.predict(x_test)
    
    MAE_train,MAE_test = error_gen(y_train,y_train_pred,y_test,y_test_pred)
    

    return r2_train,r2_test,MAE_train,MAE_test


In [ ]:
r2_train,r2_test,MAE_train,MAE_test = regresion_poli(df3,degree=3)
final = resultados.shape[0]
resultados.loc[final] = ['Regresión Polinómica','Sin procesar / Grado 3',r2_train,r2_test,MAE_train,MAE_test]

r2_train,r2_test,MAE_train,MAE_test = regresion_poli(df_bin,degree=3)
final = resultados.shape[0]
resultados.loc[final] = ['Regresión Polinómica','Agrupamiento / Grado 3',r2_train,r2_test,MAE_train,MAE_test]

r2_train,r2_test,MAE_train,MAE_test = regresion_poli(df_tf,degree=3)
final = resultados.shape[0]
resultados.loc[final] = ['Regresión Polinómica','Transf. a gausiana / Grado 3',r2_train,r2_test,MAE_train,MAE_test]

r2_train,r2_test,MAE_train,MAE_test = regresion_poli(df_tf_bin,degree=3)
final = resultados.shape[0]
resultados.loc[final] = ['Regresión Polinómica','Agrup. y transf. / Grado 3',r2_train,r2_test,MAE_train,MAE_test]
resultados

### Árbol de regresión 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

def Arbol_Regresion(df,test_size=0.2,Prec_var='price'):
    x = df.drop(Prec_var,axis=1)
    y = df[Prec_var]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=test_size)
    
    tree = DecisionTreeRegressor(min_samples_split=30,min_samples_leaf=10,random_state=0)
    tree.fit(x_train,y_train)
    
    r2_train = tree.score(x_train,y_train)
    r2_test = tree.score(x_test,y_test)
    
    y_train_pred = tree.predict(x_train)
    y_test_pred = tree.predict(x_test)
    
    MAE_train,MAE_test = error_gen(y_train,y_train_pred,y_test,y_test_pred)
    
    return r2_train,r2_test, MAE_train,MAE_test

In [ ]:
r2_train,r2_tes, MAE_train,MAE_testt = Arbol_Regresion(df3)
final = resultados.shape[0]
resultados.loc[final] = ['Árbol de regresión','Sin procesar',r2_train,r2_test, MAE_train,MAE_test]

r2_train,r2_test, MAE_train,MAE_test = Arbol_Regresion(df_bin)
final = resultados.shape[0]
resultados.loc[final] = ['Árbol de regresión','Agrupamiento',r2_train,r2_test, MAE_train,MAE_test]

r2_train,r2_test, MAE_train,MAE_test = Arbol_Regresion(df_tf)
final = resultados.shape[0]
resultados.loc[final] = ['Árbol de regresión','Transf. a gausiana',r2_train,r2_test, MAE_train,MAE_test]

r2_train,r2_test, MAE_train,MAE_test = Arbol_Regresion(df_tf_bin)
final = resultados.shape[0]
resultados.loc[final] = ['Árbol de regresión','Agrup. y transf.',r2_train,r2_test, MAE_train,MAE_test]
resultados

### Redes neuronales - Regresión

#### Compilación del modelo

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers

def get_model_compile():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32,activation='tanh',input_shape=(18,)),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)   
    ])

    optimizer=tf.keras.optimizers.Adam(0.001)
    model.compile(loss='mse',
                 optimizer=optimizer,
                 metrics=['mae','mse'])
    return model

#### División del dataset y entrenamiento del modelo

In [ ]:
x = df_tf.drop('price',axis=1)
y = df_tf['price']

batch_size = 128
epochs=500

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

model = get_model_compile()

history = model.fit(x_train,y_train,
                    epochs=epochs,batch_size=batch_size,
                    validation_data=(x_test,y_test),
                    shuffle=True,
                    verbose=0)

#### Gráfica del progreso del entrenamiento

In [ ]:
evol_train = pd.DataFrame(history.history)
evol_train['epoch'] = history.epoch

trace1 = go.Scatter(x=evol_train['epoch'], y=evol_train['mae'], mode='lines',name='mae')
trace2 = go.Scatter(x=evol_train['epoch'], y=evol_train['val_mae'], mode='lines',name='val_mae')

trace3 = go.Scatter(x=evol_train['epoch'], y=evol_train['mse'], mode='lines',name='mse')
trace4 = go.Scatter(x=evol_train['epoch'], y=evol_train['val_mse'], mode='lines',name='val_mse')

fig = make_subplots(rows=1,cols=2)
fig.add_trace(trace1,row=1,col=1)
fig.add_trace(trace2,row=1,col=1)

fig.add_trace(trace3,row=1,col=2)
fig.add_trace(trace4,row=1,col=2)

fig.update_layout(title_text="Gráficas de MAE y MSE vs Época")
fig.show()

df_tf['price_pred'] = model.predict(x)
px.scatter(df_tf,x='price',y='price_pred',title='Gráfica de los precios reales vs los precios predichos ')

In [ ]:
from sklearn.metrics import r2_score
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

r2_train = r2_score(y_train,y_pred_train)
r2_test = r2_score(y_test,y_pred_test)

MAE_TRAIN,MAE_TEST = error_gen(y_train,y_pred_train[:,0],y_test,y_pred_test[:,0])

final = resultados.shape[0]
resultados.loc[final] = ['ANN','Transf. a gausiana',r2_train,r2_test,MAE_TRAIN,MAE_TEST]
resultados